In [2]:
import cv2
import imutils
import matplotlib.pyplot as plt
import numpy as np
from sklearn.decomposition import PCA
import math
import xmltodict
import numpy as np
from skimage.feature import canny
from sklearn.metrics import confusion_matrix
import os
from pathlib import Path
import glob
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn import preprocessing
from sklearn.svm import SVC #, LinearSVC
from sklearn.neighbors import KNeighborsClassifier
import time
import copy

**Preprocessing**

In [3]:
def Preprocess(image):   
    image = cv2.resize(image,(500,500))
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    return gray,image

**Viola**


**********start of code**********

In [4]:
class Feature:
    rects=[]
    count_rects=0
    threshold=None
    left_leaf_value=None
    right_leaf_value=None
    left_node=None
    right_node=None
    found_left_leaf=None
    found_right_leaf=None
    width=None
    height=None
    
    def __init__(self,threshold,left_leaf_value,right_leaf_value,left_node,right_node,found_left_leaf,found_right_leaf,width
    ,height):
        self.threshold=threshold
        self.left_leaf_value=left_leaf_value
        self.right_leaf_value=right_leaf_value
        self.left_node=left_node
        self.right_node=right_node
        self.found_left_leaf=found_left_leaf
        self.found_right_leaf=found_right_leaf
        self.width=width
        self.height=height
        self.rects=[]
        self.count_rects=0
        
    def add_rectangle(self,x1,x2,y1,y2,weight):
        self.rects.append([float(x1),float(x2),float(y1),float(y2),float(weight)])
        self.count_rects+=1
        
    def get_direction(self,integral_image,squared_integral,i,j,scale_window,inv_no_pixels,variance):     
        sum_rectangles = 0
        for rect in self.rects:
            rx1=i + int(scale_window*rect[0])
            rx2=i + int(scale_window*(rect[0]+rect[2]))
            ry1=j + int(scale_window*rect[1])
            ry2=j + int(scale_window*(rect[1]+rect[3]))
            
            sum_rectangles += int((integral_image[rx2][ry2]-integral_image[rx1][ry2]-integral_image[rx2][ry1]+integral_image[rx1][ry1])*rect[4]);          
        mean_rectangles = sum_rectangles*inv_no_pixels
        return mean_rectangles<(self.threshold*variance) 
        

In [5]:
class Tree:
    features=[]
    
    def __init__(self):
        self.features=[]
        
    def add_feature(self,feature):
        self.features.append(feature)
        
    def get_feature_value(self,integral_image,squared_integral,i,j,scale_window,inv_no_pixels,variance):
        node = self.features[0]
        if(node.get_direction(integral_image,squared_integral,i,j,scale_window,inv_no_pixels,variance)):
            if(node.found_left_leaf):
                return node.left_leaf_value
            else:
                node = self.features[node.left_node]
        else:
            if(node.found_right_leaf):
                return node.right_leaf_value
            else:
                node = self.features[node.right_node]

In [6]:
class Stage:
    trees=[]
    threshold=None
    
    def __init__(self,threshold):
        self.threshold=threshold
        self.trees=[]

    def add_tree(self,tree):
        self.trees.append(tree)
        
    def pass_stage(self,integral_image,squared_integral,i,j,scale_window,inv_no_pixels,variance):
        sum_trees = 0
        for tree in self.trees:
            sum_trees += tree.get_feature_value(integral_image,squared_integral,i,j,scale_window,inv_no_pixels,variance)
        return sum_trees > self.threshold

In [7]:
class Viola:
    width=None
    height=None
    stages=[]
    
    def read_xml(self):
        with open('haarcascade_frontalface_default.xml', 'r', encoding='utf-8') as file:
            my_xml = file.read()
        my_dict = xmltodict.parse(my_xml)
        
        self.height= int(my_dict['opencv_storage']['cascade']['height'])
        self.width= int(my_dict['opencv_storage']['cascade']['width'])
        stageNum=int(my_dict['opencv_storage']['cascade']['stageNum'])
        
        for stage_i in range(stageNum):
            maxWeakCount=int(my_dict['opencv_storage']['cascade']['stages']['_'][stage_i]['maxWeakCount'])
            stageThreshold=float(my_dict['opencv_storage']['cascade']['stages']['_'][stage_i]['stageThreshold'])
            st= Stage(stageThreshold)
            
            stage= my_dict['opencv_storage']['cascade']['stages']['_'][stage_i]
            for tree_i in range(maxWeakCount):
                t = Tree()
                internalNodes = stage['weakClassifiers']['_'][tree_i]['internalNodes']
                leafValues = stage['weakClassifiers']['_'][tree_i]['leafValues']
                
                internalNodes = internalNodes.split(" ")
                threshold = float(internalNodes[3])
                feature_index = int(internalNodes[2])

                leafValues = leafValues.split(" ")
                left_leaf_value = float(leafValues[0])
                right_leaf_value = float(leafValues[1])

                f=Feature(threshold,left_leaf_value,right_leaf_value,-1,-1,True,True,self.width,self.height)

                rects= my_dict['opencv_storage']['cascade']['features']['_'][feature_index]['rects']['_']
                for rect_i in rects:
                    rect_i= rect_i.split(" ")
                    f.add_rectangle(*rect_i)
                
                t.add_feature(f) 
                st.add_tree(t)
                
            self.stages.append(st)
            
    def detect_faces(self,image,base_scale,inc_scale,inc_sliding,min_neighbours,do_canny_pruning,org,k):
        original = np.copy(image)
        image = np.swapaxes(image,0,1)
        # image = cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
        rects = []
        img_width = image.shape[0]
        img_height = image.shape[1]
        max_scale = min((img_width/self.width),(img_height/self.height))
        
#         Handmade Integral:
#         integral_image = np.zeros((img_width,img_height))
#         squared_integral = np.zeros((img_width,img_height))  
#         for i in range(img_width):
#             sum_curr_row=0
#             sum_curr_row2=0
#             for j in range(img_height):
#                 curr_pixel = image[i][j]
#                 if (i==0):
#                     integral_image[i][j] = sum_curr_row+curr_pixel
#                     squared_integral[i][j] = sum_curr_row2+curr_pixel*curr_pixel
#                 else:
#                     integral_image[i][j] = sum_curr_row+curr_pixel+integral_image[i-1][j]
#                     squared_integral[i][j] = sum_curr_row2+curr_pixel*curr_pixel+squared_integral[i-1][j]
                    
#                 sum_curr_row+=curr_pixel;
#                 sum_curr_row2+=curr_pixel*curr_pixel;

        integral_image, squared_integral = cv2.integral2(image)
#         integral_image= integral_image[1:,1:,]
#         squared_integral = squared_integral[1:,1:,]
        
        if (do_canny_pruning):
            canny_image = canny(image, sigma=4).astype('uint8')*255
            integral_canny, squared_canny = cv2.integral2(canny_image)
#             integral_canny = integral_canny[1:,1:,]
            
            
        scale = 1.0
        for scale in np.arange(base_scale,max_scale,scale*inc_scale):
            step = int(scale*self.width*inc_sliding)
            size = int(scale*self.width)
            inv_no_pixels = 1/(size*size)
            for i in np.arange(0,img_width-size,step):               
                for j in np.arange(0,img_height-size,step):
                    if (do_canny_pruning):
                        edges_density = integral_canny[i+size][j+size]+integral_canny[i][j]-integral_canny[i][j+size]-integral_canny[i+size][j]
                        d = edges_density/(size*size)
#                         Canny sigma 1
#                         if(d<4 or d>10):
#                             continue
#                       Canny sigma 4
                        if(d<0.2 or d>10):  #0.5,4
                            continue
                    
                    face_detected = True                  
                    sum_window=integral_image[i+size][j+size]+integral_image[i][j]-integral_image[i][j+size]-integral_image[i+size][j]
                    sum_window2=squared_integral[i+size][j+size]+squared_integral[i][j]-squared_integral[i][j+size]-squared_integral[i+size][j]
                    mean = sum_window*inv_no_pixels
                    variance = sum_window2*inv_no_pixels-(mean*mean)
                    if (variance>1):
                        variance = math.sqrt(variance)
                    else:
                        variance = 1
                        
                    for stage in self.stages:
                        if(stage.pass_stage(integral_image,squared_integral,i,j,scale,inv_no_pixels,variance)==False):
                            face_detected = False
                            break
                    if(face_detected):
                        rects.append([j,i,size,size])
                        # print("Face detected")
#                         print(i,j,size,d)

        merged_rects,_ = cv2.groupRectangles(np.asarray(rects).tolist(), min_neighbours, 0.2)
        # print(len(rects))
        print("FD",len(merged_rects))
        cropped_imgs = []
        org_imgs = []
        columns =[]
        rows=[]
        widths=[]
        if len(merged_rects)!=0:
            for rect in merged_rects:
                original = cv2.rectangle(
                    original,
                    (rect[1], rect[0]),
                    (rect[1] + rect[2], rect[0] + rect[2]),
                    (0, 255, 0),
                    2
                )

                x=rect[0]
                y=rect[1]
                w=rect[2]

    #             cropped_img=original[(rect[0]+50):(rect[0]+rect[2]-50),(rect[1]+70):(rect[1]+rect[2]-70)]
    #             cropped_img=original[(rect[0]):(rect[0]+rect[2]),(rect[1]+60):(rect[1]+rect[2]-60)]

    #             cropped_img=original[(rect[0]):(rect[0]+rect[2]),(rect[1]):(rect[1]+rect[2])]
                cropped_img=original[(rect[0]+40):(rect[0]+rect[2]-20),(rect[1]+40):(rect[1]+rect[2]-45)]
                if k!=-1:
                    cv2.imwrite('afterViola500\\'+str(k)+'.jpg',cropped_img)

    #             cropped_img= cv2.resize(cropped_img,(500,500))
                cropped_img= cv2.resize(cropped_img,(48,64))

                org=org[(rect[0]+40):(rect[0]+rect[2]-20),(rect[1]+40):(rect[1]+rect[2]-45),:]
                if k!=-1:
                    cv2.imwrite('afterViola500Color\\'+str(k)+'.jpg',org)

    #             org= imutils.resize(org, width = 500,height = 500)
                org= imutils.resize(org, width = 48,height = 64)

                cropped_imgs.append(cropped_img)
                org_imgs.append(org)
                columns.append(x)
                rows.append(y)
                widths.append(w)
            # show_images([original])
            # show_images([cropped_img])

        else:
            return cropped_imgs ,org_imgs,0,0,0
        
        return cropped_imgs ,org_imgs,columns,rows,widths
        
         

In [8]:
def viola(gray,viola,org,k):
    return viola.detect_faces(gray,5,1.25,0.1,1,True,org,k)
#     10


**********End of Code**********

**CV- Viola**

In [9]:
def viola1(gray):
    detector = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
    rects = detector.detectMultiScale(gray,scaleFactor=1.1,
                                        minNeighbors=5,
                                        minSize=(30, 30),
                                        flags=cv2.CASCADE_SCALE_IMAGE) 

    # print("rects:",len(rects))
    optional2=0
    cropped_imgs = []
    org_imgs = []
    columns =[]
    rows=[]
    widths=[]
    if (len(rects)!= 0):
        for (column, row, width, height) in rects:
            cv2.rectangle(
                gray,
                (column, row),
                (column + width, row + height),
                (0, 255, 0),
                2
            )
            # show_images([gray[row:row+width,column:column+height]])

            optional=gray[(row):(row+width),(column):(column+height)]
            optional2= cv2.resize(optional,(60,80))
            cropped_imgs.append(optional2)
            columns.append(column)
            rows.append(row)
            widths.append(width)
    else:
        return cropped_imgs,0,0,0

    return cropped_imgs,columns,rows,widths

**Utility Function**

In [10]:
def show_images(images,titles=None):
    #This function is used to show image(s) with titles by sending an array of images and an array of associated titles.
    # images[0] will be drawn with the title titles[0] if exists
    # You aren't required to understand this function, use it as-is.
    n_ims = len(images)
    if titles is None: titles = ['(%d)' % i for i in range(1,n_ims + 1)]
    fig = plt.figure()
    n = 1
    for image,title in zip(images,titles):
        a = fig.add_subplot(1,n_ims,n)
        if image.ndim == 2: 
            plt.gray()
        plt.imshow(image)
        a.set_title(title)
        plt.axis('off')
        n += 1
    fig.set_size_inches(np.array(fig.get_size_inches()) * n_ims)
    plt.show() 

**Functions for Dlib**

In [11]:
#The first utility function is rect_to_bb , short for “rectangle to bounding box”:
def rect_to_bb(rect):
	# take a bounding predicted by dlib and convert it
	# to the format (x, y, w, h) as we would normally do
	# with OpenCV
	x = rect.left()
	y = rect.top()
	w = rect.right() - x
	h = rect.bottom() - y
	# return a tuple of (x, y, w, h)
	return (x, y, w, h)

In [12]:
def shape_to_np(shape, dtype="int"):
	# initialize the list of (x, y)-coordinates
	coords = np.zeros((68, 2), dtype=dtype)
	# loop over the 68 facial landmarks and convert them
	# to a 2-tuple of (x, y)-coordinates
	for i in range(0, 68):
		coords[i] = (shape.part(i).x, shape.part(i).y)
	# return the list of (x, y)-coordinates
	return coords

In [13]:
def dlib_func(gray):
    # initialize dlib's face detector (HOG-based) and then create
    # the facial landmark predictor
    detector = dlib.get_frontal_face_detector()
    predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")
    # detect faces in the grayscale image
    rects = detector(gray, 1)
    # loop over the face detections
    for (i, rect) in enumerate(rects):
        # determine the facial landmarks for the face region, then
        # convert the facial landmark (x, y)-coordinates to a NumPy
        # array
        shape = predictor(gray, rect)
        shape = face_utils.shape_to_np(shape)
        # convert dlib's rectangle to a OpenCV-style bounding box
        # [i.e., (x, y, w, h)], then draw the face bounding box
        (x, y, w, h) = face_utils.rect_to_bb(rect)
        cv2.rectangle(gray, (x, y), (x + w, y + h), (0, 255, 0), 2)
        # show the face number
        cv2.putText(gray, "Face #{}".format(i + 1), (x - 10, y - 10),
            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
        # loop over the (x, y)-coordinates for the facial landmarks
        # and draw them on the image
        for (x, y) in shape:
            cv2.circle(gray, (x, y), 1, (0, 0, 255), -1)
    # show_images([gray])
    return gray

***Models***


In [14]:
def SVM(training_features, y_train):
#     lab_enc = preprocessing.LabelEncoder()
#     training_scores_encoded = lab_enc.fit_transform(y_train)
#     clf = LinearSVC(C=1, max_iter=8000)  #SVC(kernel='linear',c=1,)
#     clf = SVC(kernel='poly',C=1,gamma="scale")
    clf = SVC(kernel='linear',C=5,gamma="scale")
#     clf = SVC(kernel='rbf',C=4,gamma="scale")
    clf.fit(training_features, y_train)  
    return clf

In [15]:
def KNN(X_train, y_train):
    lab_enc = preprocessing.LabelEncoder()
    training_scores_encoded = lab_enc.fit_transform(y_train)
    classifier = KNeighborsClassifier(n_neighbors=3) 
    classifier.fit(X_train, training_scores_encoded)
    return classifier

In [16]:
def Randome_Forest(X_train, y_train):
    clf_Randome_Forest = RandomForestClassifier(n_estimators=800)
    clf_Randome_Forest.fit(X_train, y_train)
    return clf_Randome_Forest

# clf_Randome_Forest = pickle.load(open('random_forest.sav', 'rb'))
# result = loaded_model.score(X_test, Y_test)

In [17]:
def MLP(X_train, y_train):
    clf = MLPClassifier(random_state=1, max_iter=300).fit(X_train, y_train)
    return clf

In [18]:
# from keras.models import Sequential  
# from keras.layers import Dense, Dropout, Activation, Flatten  
# from keras.layers import Conv2D, MaxPooling2D, BatchNormalization,AveragePooling2D  
# from keras.losses import categorical_crossentropy  
# from keras.optimizers import Adam  
# from keras.regularizers import l2  
# from keras.utils import np_utils  

# def NeuralNetwork(X_train, train_y):
#     num_features = 64  
#     num_labels = 7  
#     batch_size = 64  
#     epochs = 30  
#     width, height = 48, 48 
    
#     model = Sequential()  
#     model.add(Conv2D(64, kernel_size=(3, 3), activation='relu', input_shape=(X_train.shape[1:])))  
#     model.add(Conv2D(64,kernel_size= (3, 3), activation='relu'))  
#     # model.add(BatchNormalization())  
#     model.add(MaxPooling2D(pool_size=(2,2), strides=(2, 2)))  
#     model.add(Dropout(0.5))  

#     #2nd convolution layer  
#     model.add(Conv2D(64, (3, 3), activation='relu'))  
#     model.add(Conv2D(64, (3, 3), activation='relu'))  
#     # model.add(BatchNormalization())  
#     model.add(MaxPooling2D(pool_size=(2,2), strides=(2, 2)))  
#     model.add(Dropout(0.5))  

#     #3rd convolution layer  
#     model.add(Conv2D(128, (3, 3), activation='relu'))  
#     model.add(Conv2D(128, (3, 3), activation='relu'))  
#     # model.add(BatchNormalization())  
#     model.add(MaxPooling2D(pool_size=(2,2), strides=(2, 2)))  

#     model.add(Flatten())  

#     #fully connected neural networks  
#     model.add(Dense(1024, activation='relu'))  
#     model.add(Dropout(0.2))  
#     model.add(Dense(1024, activation='relu'))  
#     model.add(Dropout(0.2))  

#     model.add(Dense(num_labels, activation='softmax'))  

#     # model.summary()  

#     #Compliling the model  
#     model.compile(loss=categorical_crossentropy,  
#                   optimizer=Adam(),  
#                   metrics=['accuracy'])  

#     #Training the model  
#     model.fit(X_train, train_y,  
#               batch_size=batch_size,  
#               epochs=epochs,  
#               verbose=1,  
#               validation_data=(X_test, test_y),  
#               shuffle=True)  


**Reading Datasets**

In [19]:
read_original = 0

In [20]:
def collect_paths_to_files(dataset_dir):
    labels_train=[]
    images_train=[]
    labels_test=[]
    images_test=[]
    emotion_dict={
        "anger":0,
        "disgust":1,
        "fear":2,
        "happiness":3,
        "neutral":4,
        "sadness":5,
        # "surprise":6
    }
    # emotion_dict={
    #     "anger":0,
    #     "disgust":0,
    #     "fear":0,
    #     "happiness":1,
    #     "neutral":1,
    #     "sadness":1,
    #     # "surprise":6
    # }
    # emotion_dict={
    #     "anger":0,
    #     "disgust":0,
    #     "fear":0,
    #     "happiness":1,
    #     "neutral":1,
    #     "sadness":1,
    #     # "surprise":6
    # }
    dataset_dir = Path(dataset_dir)
    folders_1 = sorted(glob.glob (str(dataset_dir)+"/*"))
    # print("1",folders_1)
    fold_num=0
    num_train = 0
    num_test = 0
    for folder3 in folders_1:
        # print(folder3)
        # for folder2 in folder3:
            # print(folder2)
        # for folder2 in folders_1:
        fold_num+=1
        emotions = sorted(glob.glob(str(folder3)+"\\"+str(folder3).split("\\")[-1]+ "/*"))
        # print("emotion",emotions)
        for emotion in emotions:
            dir = emotion.split('\\')
            e= dir[-1]
            # print("e",e)
            if e != 'mixed' and e != 'extra' and e!='surprise' and e!='neutral':
                if os.path.isdir(emotion):
                    takes = sorted(glob.glob (emotion+'/*'))
                    # print("youmna",takes)
                    for take in takes:
                        curr = sorted(glob.glob (take+'/*.jpg'))
                        i=0
                        # print("fo2 image")
                        for image in curr:
                            # if read_original==0:
                            #     curr = sorted(glob.glob (take+'/*.jpg'))
                            #     for image in curr: 
                            #         img = cv2.imread(r'trainingSet\\'+e+'\\take000\\'+str(num_train)+'.jpg')
                            #         images_test.append(img)
                            #         labels_test.append(emotion_dict[e])
                                    
                            if i>int(len(curr)/2) and i<=int((len(curr)/2)+1):
                                # print("ta7t if")
                                if fold_num<45:
#                                 if fold_num==1:
                                    if read_original == 0:
                                        # print("sdhnmkjf")
                                        img = cv2.imread(image) 
                                        cv2.imwrite('trainingSet\\'+e+'\\take000\\'+str(num_train)+'.jpg',img)
                                    else:
                                        # print("elsee fo2")
                                        img = cv2.imread(r'trainingSet\\'+e+'\\take000\\'+str(num_train)+'.jpg')
                                        show_images([img])
                                    num_train+=1
                                    images_train.append(img)
                                    labels_train.append(emotion_dict[e])
                                else:
                                    # print("elsee ta7t")
                                    if read_original == 0:
                                        img = cv2.imread(image) 
                                        cv2.imwrite('testSet\\'+e+'\\take000\\'+str(num_test)+'.jpg',img)
                                    else:
                                        img = cv2.imread(r'testSet\\'+e+'\\take000\\'+str(num_test)+'.jpg')
                                    images_test.append(img)
                                    labels_test.append(emotion_dict[e])
                                    num_test+=1
                            i=i+1

    return labels_train,images_train,labels_test,images_test

In [21]:
def read_stored_imgs():
    labels_train=[]
    images_train=[]
    labels_test=[]
    images_test=[]
    emotion_dict={
        "anger":0,
        "disgust":1,
        "fear":2,
        "happiness":3,
        "neutral":4,
        "sadness":5,
        # "surprise":6
    }
    # emotion_dict={
    #     "anger":0,
    #     "disgust":0,
    #     "fear":0,
    #     "happiness":1,
    #     "neutral":1,
    #     "sadness":1,
    #     # "surprise":6
    # }

    # emotion_dict={
    #     "anger":0,
    #     "disgust":0,
    #     "fear":0,
    #     "happiness":1,
    #     "neutral":1,
    #     "sadness":1,
    #     # "surprise":6
    # }
    path_of_the_directory="C:\\Users\\farah\\Desktop\\Image project\\trainingSet"
    # path_of_the_directory="C:\\Users\\farah\\Desktop\\Image project\\trainingSet"
    for filename in os.listdir(path_of_the_directory):
        emotions = os.path.join(path_of_the_directory,filename)
        # print(emotions)
        path=emotions+'\\take000\\'
        for images in os.listdir(path):
            image = os.path.join(path,images)
            img = cv2.imread(image)
            # show_images([img])
            images_train.append(img)
            labels_train.append(emotion_dict[filename])

    path_of_the_directory="C:\\Users\\farah\\Desktop\\Image project\\testSet"
    for filename in os.listdir(path_of_the_directory):
        emotions = os.path.join(path_of_the_directory,filename)
        # print(emotions)
        path=emotions+'\\take000\\'
        for images in os.listdir(path):
            image = os.path.join(path,images)
            img = cv2.imread(image)
            # show_images([img])
            images_test.append(img)
            labels_test.append(emotion_dict[filename])
    return labels_train,images_train,labels_test,images_test


In [22]:
if (read_original==0):
    labels_train,images_train,labels_test,images_test=collect_paths_to_files("Y:\\GP\\MUG\\subjects3")
    # labels_train,images_train,labels_test,images_test=collect_paths_to_files("C:\\Users\\farah\\Desktop\\Image project\\temp")
else:
    labels_train,images_train,labels_test,images_test=read_stored_imgs()
    
# labels_train,images_train,labels_test,images_test=collect_paths_to_files("C:\\Users\\PC\\Desktop\\Semester 9\\Image\\Project\\sowar")


In [23]:
print("size of test",len(labels_test))
print("size of train",len(labels_train))
labels_dataset = copy.deepcopy(labels_train)
images_dataset = copy.deepcopy(images_train)
for i in range (len(labels_test)):
    labels_dataset.append(labels_test[i])
    images_dataset.append(images_test[i])

size of test 108
size of train 600


In [30]:
for i in range(len(labels_test)):
    # image=cv2.imread(os.path.join(DIR, i))
    # blurred_image = motion_blur(image)
    cv2.imwrite('Y:/GP/labelTest/'+str(labels_test[i])+'_'+str(i)+'.png',images_test[i]) 
    # count+=1

In [31]:
with open('labelTest.txt', 'w') as file:
    for label in labels_test:
        file.write("%i\n" % label)

**Gabor**

In [23]:
def gaborFun(image):
    features = []
    sizes=[5,9,11,15,19]
    phi=0
    gamma = 0.5
    sigma = 5
    lam = np.pi/4.
    for theta in range (8): #8
        theta=theta/4.*np.pi
        for i in range (len(sizes)):
            kernel=cv2.getGaborKernel((sizes[i], sizes[i]), sigma, theta, lam, gamma, phi, ktype=cv2.CV_32F)
            fimg = cv2.filter2D(image, cv2.CV_8UC3, kernel)    
            filteredImage=fimg.reshape(-1)
            features.append(filteredImage)
    return features

**Eigen Function**

In [24]:
def eigenFun(X_train):
    # images_grayscale=[]
    # for i in range(len(X_train)):
    #     # image_gray = color.rgb2gray(X_train[i])
    #     image_gray = X_train[i].flatten()
    #     # print(image_gray.shape)
    #     # image_gray=image_gray.reshape(image_gray.shape[0],1)
    #     # print(image_gray)
    #     images_grayscale.append(image_gray)
    # print(images_grayscale[0].shape)
    # plt.figure(figsize=(18, 7))
    # print("list el gray scale",len(images_grayscale))
    # print("shape el image el wa7da",images_grayscale[0].shape)
    # images_grayscale=np.array(images_grayscale)
    # print("shape el image ",images_grayscale.shape)
    # pca = PCA().fit(X_train)
    # plt.plot(pca.explained_variance_ratio_.cumsum(), lw=3)
    # print((np.where(pca.explained_variance_ratio_.cumsum() > 0.95)))

    # pca = PCA(n_components=(np.where(pca.explained_variance_ratio_.cumsum() > 0.80))[0][0]).fit(X_train)
    pca = PCA(n_components=90).fit(X_train)
    
    return pca

In [25]:
def eigenFunGabor(X_train):
    # images_grayscale=[]
    # for i in range(len(X_train)):
    #     # image_gray = color.rgb2gray(X_train[i])
    #     image_gray = X_train[i].flatten()
    #     # print(image_gray.shape)
    #     # image_gray=image_gray.reshape(image_gray.shape[0],1)
    #     # print(image_gray)
    #     images_grayscale.append(image_gray)
    # print(images_grayscale[0].shape)
    # plt.figure(figsize=(18, 7))
    # print("list el gray scale",len(images_grayscale))
    # print("shape el image el wa7da",images_grayscale[0].shape)
    # images_grayscale=np.array(images_grayscale)
    # print("shape el image ",images_grayscale.shape)
    # pca = PCA().fit(X_train)
    # plt.plot(pca.explained_variance_ratio_.cumsum(), lw=3)
    # print((np.where(pca.explained_variance_ratio_.cumsum() > 0.95)))

    # pca = PCA(n_components=(np.where(pca.explained_variance_ratio_.cumsum() > 0.97))[0][0]).fit(X_train)
    pca = PCA(n_components=270).fit(X_train)
    
    return pca

**Extra trials for Eigen Faces**

In [26]:
def eigen_faces(train):
    images_grayscale=[]
    for i in range(len(train)):
        image_gray = color.rgb2gray(train[i])
        image_gray = image_gray.reshape(-1)
        images_grayscale.append(image_gray)
#########################PCA
    M=len(images_grayscale)
    print("length of images",M)
    mean_of_training_set = np.mean(images_grayscale, axis=0)
    print("mean.shape",mean_of_training_set.shape)
    subtracted_list = []
    for i in range (M):
        x=images_grayscale[i]-mean_of_training_set #Subtract the calculated mean from the vector of training image to find subtracted mean
        subtracted_list.append(x) #Indicates the difference between the vector of m’th training image and the mean image.
    sigma = (np.var(images_grayscale, axis=0))**0.5
    normalized_X = (images_grayscale-mean_of_training_set)/sigma
    print("normaalized", normalized_X.shape)
    subtracted=np.array(subtracted_list).T
    print("subtracted_list",len(subtracted_list))   
    print("subtracted_list",subtracted_list[0].shape)   
    print("subtracted",subtracted.shape) #(8,313600)        
    subtracted_transpose=subtracted.T
    print("subtracted_transpose",subtracted_transpose.shape)
    print("normaalized", normalized_X)
    print("subtracted",subtracted_transpose)  
    covariance=np.dot(normalized_X,normalized_X.T)/(normalized_X.T).shape[0]
    # covariance=np.dot(subtracted_transpose,subtracted)/(subtracted).shape[0]
    # cov = np.dot(X.T, X)/X.shape[0]
    # _, eigenvalues, eigenvectors = np.linalg.svd(covariance, full_matrices=False)

    # C = np.dot(subtracted,eigenvectors[1])
    # C1=np.dot(subtracted_transpose,C)
    # lam = np.dot(eigenvalues[1],eigenvectors[1])
    # return covariance, eigenvalues, eigenvectors, C1, lam

**Feature Extraction**

In [27]:
read=0

In [28]:
def extract_features_training(trainX,labelsX,viola_inst):
    features = []
    mxm = len(labelsX)
    trainAfterPreProcess=[]
    labelsOut=[]
    for i in range(mxm):
        # print(i)
        image = trainX[i]
        if(read==0):  
            gray_out,image_out1=Preprocess(image)
            # gray_out2,image_out2,_,_,_=viola(gray_out,viola_inst,image_out1,i)
            gray_out2,x,y,w=viola1(gray_out)
            if (len(gray_out2)!=0):
                cv2.imwrite('afterViola\\'+str(i)+'.jpg',gray_out2[0])
                # cv2.imwrite('afterViolaOrg\\'+str(i)+'.jpg',image_out2[0])
                
        else:
            gray_out2 = [cv2.imread(r'afterViolaKFold\\'+str(i)+'.jpg',cv2.IMREAD_GRAYSCALE)]
            # image_out2 = [cv2.imread(r'afterViolaOrg\\'+str(i)+'.jpg')]
        if (len(gray_out2)!=0):
            image_out3 = gray_out2[0]
            flattend_image=image_out3.flatten()
            trainAfterPreProcess.append(flattend_image)
            curr=gaborFun(image_out3)
            curr=np.array(curr)
            curr=curr.flatten()   
            features.append(curr)
            # print("curr shape=",curr.shape)
            labelsOut.append(labelsX[i])

    features= np.array(features)  
    pca=eigenFun(trainAfterPreProcess)#(8,5)
    X_train_pca = pca.transform(trainAfterPreProcess)
    # X_train_pca = trainAfterPreProcess
    print("X_train_pca",X_train_pca.shape) 

    
    pcaGabor=eigenFunGabor(features)
    gabor_pca = pcaGabor.transform(features)
    # gabor_pca = features
    print("gabor_pca",gabor_pca.shape)  
    f=np.append(gabor_pca,X_train_pca,axis=1)
    print("f=",f.shape) 
    return f, pca,pcaGabor,labelsOut #pcaGabor

In [29]:
read_test=0

In [30]:
def extract_features_test(test, pca,pcaGabor,viola_inst,k):
    features = []
    f=[]
    image = test
    if(read_test==0):  
        gray_out,image_out1=Preprocess(image)
        # gray_out2,image_out2,x,y,w=viola(gray_out,viola_inst,image_out1,-1)
        gray_out2,x,y,w=viola1(gray_out)
        if(len(gray_out2)!=0):
            if k!=-1:
                cv2.imwrite('afterViolaTest\\'+str(k)+'.jpg',gray_out2[0])
                # cv2.imwrite('afterViolaOrgTest\\'+str(k)+'.jpg',image_out2[0])
                        
    else:
        gray_out2=[]
        image_out2=[]
        gray_out2.append(cv2.imread(r'afterViolaTest\\'+str(k)+'.jpg',cv2.IMREAD_GRAYSCALE))
        # print(len(gray_out2))
        # image_out2.append(cv2.imread(r'afterViolaOrgTest\\'+str(k)+'.jpg'))

    testAfterPreProcess=[]
    # print(len(gray_out2))
    if(len(gray_out2)!=0):
        for face in gray_out2:
            # print("IN")
            # print(type(face))
            image_out3 =np.array(face)
            # print(image_out3.shape)
            # print(type(image_out3))
            flattend_image=image_out3.flatten()
            # print("flattened",flattend_image.shape)
            testAfterPreProcess.append(flattend_image)
            curr=gaborFun(image_out3)
            curr=np.array(curr)
            curr=curr.flatten()   
            features.append(curr)
            # print(curr.shape)
        
        features= np.array(features)  
        # print("curr=",curr.shape)
        # print("features=",features.shape)
        X_test_pca = pca.transform(testAfterPreProcess)
        # X_test_pca = testAfterPreProcess
        print("X_test_pca=",X_test_pca.shape)


        gabor_pca_test = pcaGabor.transform(features)
        # gabor_pca_test = features
        # print("gabor_pca_test",gabor_pca_test.shape) 
        f=np.append(gabor_pca_test,X_test_pca,axis=1)
    
        print("f=",f.shape) 
    if (read_test==1):
        return f,0,0,0
    
    return f,x,y,w

***TRAINING***

In [31]:
viola_inst = Viola()
viola_inst.read_xml()
start_time = time.time()
features,pca,pcaGabor,labels_train=extract_features_training(images_train,labels_train,viola_inst)
print("--- %s seconds ---" % (time.time() - start_time))
clf=SVM(features,labels_train)
# clf=KNN(features,labels_train)
# clf=Randome_Forest(features,labels_train)
# clf=MLP(features,labels_train)

X_train_pca (600, 90)
gabor_pca (600, 270)
f= (600, 360)
--- 92.98318409919739 seconds ---


***TESTING [Without Camera]***

In [32]:
svm_predictions=[]
k=0
for img in images_test:
    test_point,_,_,_=extract_features_test(img,pca,pcaGabor,viola_inst,k+len(labels_train))
    if(len(test_point)!=0):
        res=clf.predict(test_point)
        print("prediction= ",res)
        print("Actual= ",labels_test[k])
        svm_predictions.append(res[0])
        # show_images([img])
    k+=1

        
# results_SVM = np.array([svm_predictions[0] == labels_test])  
# accuracy_SVM= results_SVM[results_SVM==True].shape[0]/np.array(labels_test).shape[0]

# print(svm_predictions)
# print(labels_test)
# print(results_SVM)
from sklearn.metrics import accuracy_score
accuracy_SVM2 = accuracy_score(labels_test, svm_predictions)


# print("SVM Classifier Accuracy: ", accuracy_SVM*100, "%")
print("SVM Classifier Accuracy: ", accuracy_SVM2*100, "%")
    

X_test_pca= (1, 90)
f= (1, 360)
prediction=  [0]
Actual=  0
X_test_pca= (1, 90)
f= (1, 360)
prediction=  [0]
Actual=  0
X_test_pca= (1, 90)
f= (1, 360)
prediction=  [2]
Actual=  0
X_test_pca= (1, 90)
f= (1, 360)
prediction=  [1]
Actual=  1
X_test_pca= (1, 90)
f= (1, 360)
prediction=  [1]
Actual=  1
X_test_pca= (1, 90)
f= (1, 360)
prediction=  [1]
Actual=  1
X_test_pca= (1, 90)
f= (1, 360)
prediction=  [1]
Actual=  1
X_test_pca= (1, 90)
f= (1, 360)
prediction=  [1]
Actual=  1
X_test_pca= (1, 90)
f= (1, 360)
prediction=  [2]
Actual=  2
X_test_pca= (1, 90)
f= (1, 360)
prediction=  [2]
Actual=  2
X_test_pca= (1, 90)
f= (1, 360)
prediction=  [3]
Actual=  3
X_test_pca= (1, 90)
f= (1, 360)
prediction=  [0]
Actual=  3
X_test_pca= (1, 90)
f= (1, 360)
prediction=  [3]
Actual=  3
X_test_pca= (1, 90)
f= (1, 360)
prediction=  [5]
Actual=  5
X_test_pca= (1, 90)
f= (1, 360)
prediction=  [5]
Actual=  5
X_test_pca= (1, 90)
f= (1, 360)
prediction=  [5]
Actual=  5
X_test_pca= (1, 90)
f= (1, 360)
predicti

In [33]:
print("SVM Classifier Accuracy: ", accuracy_SVM2*100, "%")


SVM Classifier Accuracy:  86.11111111111111 %


***TESTING [With Camera]***

In [34]:
# svm_predictions=[]
# k=0
# # url = "http://10.1.0.144:8080/video"
# url = "http://192.168.58.58:8080/video"
# cap = cv2.VideoCapture(0) # 0--> Webcam  url-->from mobile
# emotion_dict_reversed={
#         0:"anger",
#         1:"disgust",
#         6:"fear",
#         3:"happiness",
#         4:"neutral",
#         5:"sadness",
#         6:"surprise"
#     }
# read_test=0
# while(True):
#     camera, frame = cap.read()
#     if frame is not None:
#         frame=cv2.resize(frame,(500,500))
#         test_point,x,y,w=extract_features_test(frame,pca,pcaGabor,viola_inst,-1)
#         if(len(test_point)!=0):
#             for i in range(test_point.shape[0]):
#                 res=clf.predict(test_point[i].reshape(1,-1))
#                 res=res[0]
#                 print("prediction= ",emotion_dict_reversed[res])
#                 cv2.rectangle(frame,(x[i],y[i]), (x[i]+w[i],y[i]+w[i]), (255,0,0), thickness=3)
#                 cv2.putText(frame, emotion_dict_reversed[res] ,(int(x[i]+w[i]), int(y[i]+w[i])),cv2.FONT_HERSHEY_SIMPLEX,1,(0,255,0),2)

#         cv2.imshow("Frame", frame)

#     if cv2.waitKey(10) == ord('q'):
#         break

# # # When everything is done, release the capture
# cap.release()
# cv2.destroyAllWindows()


**Computing The Confusion Matrix**

In [35]:
confusion_matrix_SVM = confusion_matrix(labels_test, svm_predictions)
print("SVM Confusion Matrix: ")
print(confusion_matrix_SVM)


SVM Confusion Matrix: 
[[15  2  3  0  4]
 [ 0 22  0  0  0]
 [ 0  0 13  0  2]
 [ 1  1  0 23  0]
 [ 0  0  1  1 20]]


**Comparision Metrics**

Computing K-Fold

In [36]:
#Kfold 
from sklearn.model_selection import cross_val_score
# clf_cross_validation = MLPClassifier(random_state=1, max_iter=300)
clf_cross_validation = SVC(kernel='linear',C=5,gamma="scale")#SVC(gamma='scale',C=1000)
# features_dataset=extract_features_training()
features_dataset,pca_dataset,pcaGabor_dataset,labels_train_dataset=extract_features_training(images_dataset,labels_dataset,viola_inst)
scores = cross_val_score(clf_cross_validation, features_dataset, labels_train_dataset, cv=5)

print((np.sum(scores)/5)*100)

X_train_pca (708, 90)
gabor_pca (708, 270)
f= (708, 360)
79.8072120667266


In [37]:
print((np.sum(scores)/5)*100)

79.8072120667266


Computing F1-score

In [38]:
#F1-score
from sklearn.metrics import f1_score, make_scorer
from sklearn.model_selection import GridSearchCV
# labels_test, svm_predictions
print(len(svm_predictions))

print("F1-score on testing data for SVM: {:.4f}%".format(f1_score(labels_test, np.array(svm_predictions),average='weighted')*100))
# print("F1-score on testing data for Random Forest: {:.4f}%".format(f1_score(labels_test, np.array(svm_predictions),average='weighted')*100))

108
F1-score on testing data for SVM: 85.7277%


In [39]:
precision =[]
recall =[]
for i in range(confusion_matrix_SVM.shape[0]):
    precision.append(confusion_matrix_SVM[i][i]*100/sum(confusion_matrix_SVM[i]))
    recall.append(confusion_matrix_SVM[i][i]*100/np.sum(confusion_matrix_SVM,axis=0)[i])


# print("Stress Precision : ",precision[0] , "Recall : ", recall[0])
# print("Not stress Precision : ",precision[1] , "Recall : ", recall[1])
# print("Fear Precision : ",precision[2] , "Recall : ", recall[2] )
# print("Happiness Precision : ",precision[3] , "Recall : ", recall[3] )
# print("Sadness Precision : ",precision[4] , "Recall : ", recall[4] )

print("Anger Precision : ",precision[0] , "Recall : ", recall[0])
print("Disgust Precision : ",precision[1] , "Recall : ", recall[1])
print("Fear Precision : ",precision[2] , "Recall : ", recall[2] )
print("Happiness Precision : ",precision[3] , "Recall : ", recall[3] )
print("Sadness Precision : ",precision[4] , "Recall : ", recall[4] )


Anger Precision :  62.5 Recall :  93.75
Disgust Precision :  100.0 Recall :  88.0
Fear Precision :  86.66666666666667 Recall :  76.47058823529412
Happiness Precision :  92.0 Recall :  95.83333333333333
Sadness Precision :  90.9090909090909 Recall :  76.92307692307692


In [40]:
# [[15  2  4  0  3]     
#  [ 0 22  0  0  0]
#  [ 0  0 13  0  2]
#  [ 1  0  0 24  0]
#  [ 0  0  2  0 20]]